In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [6]:
import hopsworks
import pandas as pd
from datetime import timedelta
from src.inference import get_feature_store, fetch_predictions

def fetch_hourly_rides(hours):
    current_hour = (pd.Timestamp.now(tz="Etc/UTC") - timedelta(hours=hours)).floor('h')
    current_hour = current_hour.replace(year=2024)

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_NAME,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour >= current_hour)

    return query.read(read_options={"arrow_flight_config": {"timeout": 600}})

In [14]:
df = fetch_hourly_rides(1)

2025-05-10 17:50:13,127 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 17:50:13,155 INFO: Initializing external client
2025-05-10 17:50:13,155 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 17:50:14,126 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214680
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (25.01s) 


In [15]:
df

,pickup_hour,pickup_location_id,rides
0,2024-12-03 08:00:00+00:00,3211.06,0
1,2024-06-22 00:00:00+00:00,4137.14,0
2,2024-10-17 08:00:00+00:00,4425.02,7
3,2024-07-24 01:00:00+00:00,4007.01,0
4,2024-11-20 23:00:00+00:00,3759.07,1
...,...,...,...
3214009,2024-07-09 09:00:00+00:00,3931.09,0
3214010,2024-07-01 23:00:00+00:00,3657.06,2
3214011,2024-06-21 18:00:00+00:00,3266.01,0
3214012,2024-07-24 02:00:00+00:00,3520.01,0


In [11]:
df_pred = fetch_predictions(1)

2025-05-10 17:48:42,256 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 17:48:42,265 INFO: Initializing external client
2025-05-10 17:48:42,266 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 17:48:42,868 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214680
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.73s) 


In [12]:
df_pred

,pickup_location_id,predicted_demand,pickup_hour
0,4710.06,9.0,2024-05-10 21:00:00+00:00
1,3696.05,0.0,2024-05-10 21:00:00+00:00
2,4467.03,3.0,2024-05-10 21:00:00+00:00
3,4310.06,2.0,2024-05-10 21:00:00+00:00
4,3981.1,0.0,2024-05-10 21:00:00+00:00
...,...,...,...
1131,2782.02,0.0,2024-05-10 22:00:00+00:00
1132,2923.01,0.0,2024-05-10 22:00:00+00:00
1133,3839.02,2.0,2024-05-10 22:00:00+00:00
1134,4416.10,2.0,2024-05-10 22:00:00+00:00


In [16]:
merged_df = pd.merge(df, df_pred, on=['pickup_location_id', 'pickup_hour'])

In [17]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand
0,2024-05-10 21:00:00+00:00,4095.03,0,1.0
1,2024-05-10 22:00:00+00:00,3803.09,3,5.0
2,2024-05-10 22:00:00+00:00,3850.03,1,1.0
3,2024-05-10 22:00:00+00:00,4392.04,5,2.0
4,2024-05-10 22:00:00+00:00,3420.01,1,3.0
...,...,...,...,...
1131,2024-05-10 21:00:00+00:00,3394.03,0,1.0
1132,2024-05-10 22:00:00+00:00,2872.02,0,-0.0
1133,2024-05-10 21:00:00+00:00,3070.04,1,1.0
1134,2024-05-10 21:00:00+00:00,3547.04,0,1.0


In [18]:
merged_df.sort_values(["pickup_location_id", "pickup_hour"])

,pickup_hour,pickup_location_id,rides,predicted_demand
156,2024-05-10 21:00:00+00:00,190 Morgan,0,-0.0
506,2024-05-10 22:00:00+00:00,190 Morgan,0,-0.0
461,2024-05-10 21:00:00+00:00,2733.03,0,-0.0
422,2024-05-10 22:00:00+00:00,2733.03,0,-0.0
97,2024-05-10 21:00:00+00:00,2782.02,0,1.0
...,...,...,...,...
851,2024-05-10 22:00:00+00:00,4743.04,0,1.0
242,2024-05-10 21:00:00+00:00,4748.07,0,5.0
177,2024-05-10 22:00:00+00:00,4748.07,1,5.0
247,2024-05-10 21:00:00+00:00,4751.01,1,1.0


In [19]:
import pandas as pd  
import plotly.express as px
df1 = df
df2 = df_pred

# Merge the DataFrames on 'pickup_location_id' and 'pickup_hour'  
merged_df = pd.merge(df1, df2, on=['pickup_location_id', 'pickup_hour'])  

# Calculate the absolute error  
merged_df['absolute_error'] = abs(merged_df['predicted_demand'] - merged_df['rides'])  

# Group by 'pickup_hour' and calculate the mean absolute error (MAE)  
mae_by_hour = merged_df.groupby('pickup_hour')['absolute_error'].mean().reset_index()  
mae_by_hour.rename(columns={'absolute_error': 'MAE'}, inplace=True)  

# Create a Plotly plot  
fig = px.line(  
    mae_by_hour,  
    x='pickup_hour',  
    y='MAE',  
    title='Mean Absolute Error (MAE) by Pickup Hour',  
    labels={'pickup_hour': 'Pickup Hour', 'MAE': 'Mean Absolute Error'},  
    markers=True  
)  

# Show the plot  
fig.show()